# Generating Multiple Images

### Check GPU

In [ ]:
!nvidia-smi
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

## Install Requirements

In [ ]:
!pip install -q --upgrade accelerate transformers ftfy
!pip install -q git+https://github.com/huggingface/diffusers

In [ ]:
!wget https://rhods-public.s3.amazonaws.com/src/xformers-0.0.21.dev584-cp39-cp39-manylinux2014_x86_64.whl -O xformers-0.0.21.dev584-cp39-cp39-manylinux2014_x86_64.whl
!pip install -q xformers-0.0.21.dev584-cp39-cp39-manylinux2014_x86_64.whl

In [ ]:
!pip list | grep -e torch -e torchvision -e diffusers -e accelerate -e torchvision -e transformers -e ftfy -e tensorboard -e Jinja2 -e xformers -e protobuf


## Settings

In [ ]:
import os
from datetime import datetime

VERSION = os.environ.get("VERSION", f"notebook-output")

MODEL_ID = "cfchase/stable-diffusion-rhteddy"
# MODEL_ID = "runwayml/stable-diffusion-v1-5"
# MODEL_ID = os.path.join(os.getcwd(), f"{VERSION}/stable_diffusion_weights/redhat-dog")

print(f"Weights loaded from be saved at {MODEL_ID}")


## Test Inference

In [ ]:
import torch
from diffusers import DiffusionPipeline

pipeline = DiffusionPipeline.from_pretrained(MODEL_ID)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pipeline.to(device)
pipeline.enable_xformers_memory_efficient_attention()


In [ ]:
from datetime import datetime
from torch import autocast

prompt = "photo of a rhteddy dog on the beach"
negative_prompt = ""
num_samples = 6
guidance_scale = 7.5
num_inference_steps = 200
height = 512
width = 512

generator = torch.Generator(device=device)
generator.seed()

with autocast(device.type), torch.inference_mode():
    images = pipeline(
        prompt,
        height=height,
        width=width,
        negative_prompt=negative_prompt,
        num_images_per_prompt=num_samples,
        num_inference_steps=num_inference_steps,
        guidance_scale=guidance_scale,
        generator=generator
    ).images


date = datetime.now()
date_string = date.strftime('%Y%m%d%H%M%S')
IMG_DIR =  os.path.join(os.getcwd(), f"{VERSION}/generated-images/{date_string}")
os.makedirs(IMG_DIR, exist_ok=True)

for i, img in enumerate(images):
    img.save(os.path.join(IMG_DIR, f"{i}.jpg"))


In [ ]:
import os
import math
import matplotlib.pyplot as plt
from PIL import Image

directory = IMG_DIR
images = []
for filename in os.listdir(directory):
    if filename.endswith('.jpg'):
        img = Image.open(os.path.join(directory, filename))
        images.append(img)

num_show = min(len(images), 12)
n_cols = 3
n_rows = math.ceil(len(images) / n_cols)
scale = 4
fig, axs = plt.subplots(n_rows, n_cols, figsize=(n_cols * scale, n_rows * scale),  gridspec_kw={'hspace': 0, 'wspace': 0})
for i in range(n_rows):
    for j in range(n_cols):
        axs[i, j].axis('off')
        x = i * n_cols + j
        if x < len(images):
            axs[i, j].imshow(images[x])
plt.show()
